# Crashcourse Methods
When solving structural models numerically, we need to teach the computer how to perform certain calculations. Many techniques have been proven to be very useful and are incorporated in toolboxes such as [econ-ark](https://econ-ark.org/), [quantecon](https://quantecon.org/), or [ssj](https://github.com/shade-econ/sequence-jacobian). Using these toolboxes might seem as a blackbox at first, meaning having an understanding of how the techniques work is crucial to assess results and adapt or debug code.

To de-mystify the blackbox of such toolboxes, I want to highlight 4 important concepts:
- Backward iteration
- Calculating expectations
- Endogeneous Gridpoint Method (EGM)
- Constructing Functions

For more detailed information on how we do things in HARK, have a look at our [notebooks](https://econ-ark.org/materials/) or look at a great description of solving [dyamic stochastic problems](https://github.com/llorracc/SolvingMicroDSOPs).

## Part 1: Mathematical background

Let us start by stating a standard consumption-saving problem.

A consumer wants to solve the following problem:
\begin{align*}
v_t(m_t) &= \max_{c_t} {~} u(c_t) + \beta (1-D_{t+1})  \mathbb{E}_{t} \left[ (\Gamma_{t+1}\psi_{t+1})^{1-\rho} v_{t+1}(m_{t+1}) \right], \\
a_t &= m_t - c_t, \\
a_t &\geq \text{$\underline{a}$}, \\
m_{t+1} &= R/(\Gamma_{t+1} \psi_{t+1}) a_t + \theta_{t+1}, \\
(\psi_{t+1},\theta_{t+1}) &\sim F_{t+1}, \\
\mathbb{E}[\psi]&=\mathbb{E}[\theta] = 1, \\
u(c) &= \frac{c^{1-\rho}}{1-\rho}.
\end{align*}

That is, the agent maximizes lifetime utility $v_t(m_t)$ where $m_t$ is the state variable 'beginning-of-period market resources' which consists of last-periods assets (savings) including the earned return and labor income. Note that labor income in HARK consists of a permanent ($\psi$) and a transitory ($\theta$) shock and NOT a persistent component. We do this to normalize all variables by permanent income and save one state variable. Additionally, we allow for a permanent growth factor of income $\Gamma$. Lastly, the agent discounts the future with a discount factor $\beta$ and has a probability of dying $D$.

The agent chooses then how much to consume $c_t$ and how much to save $a_t$. These are the polchoice variables.

### Taking First order Conditions

First, we rewrite the problem by substituting $m_{t+1}$ with the transition equation. Now, the second term of the reformulated maximand represents "end of period value of assets" $\mathfrak{v}_t(a_t)$ ("Gothic v"):

\begin{align*}
v_t(m_t) &= \max_{c_t} {~} u(c_t) + \underbrace{\beta (1-D_{t+1})  \mathbb{E}_{t} \left[ (\Gamma_{t+1}\psi_{t+1})^{1-\rho} v_{t+1}(R/(\Gamma_{t+1} \psi_{t+1}) a_t + \theta_{t+1}) \right]}_{\equiv \mathfrak{v}_t(a_t)}.
\end{align*}

The first order condition with respect to $c_t$ is thus simply:

\begin{align*}
u^{\prime}(c_t) - \mathfrak{v}'_t(a_t) = 0 \Longrightarrow c_t^{-\rho} = \mathfrak{v}'_t(a_t) \Longrightarrow c_t = \mathfrak{v}'_t(a_t)^{-1/\rho},
\end{align*}

and the marginal value of end-of-period assets can be computed as:

\begin{align*}
\mathfrak{v}'_t(a_t) = \beta (1-D_{t+1})  \mathbb{E}_{t} \left[ R (\Gamma_{t+1}\psi_{t+1})^{-\rho} v'_{t+1}(R/(\Gamma_{t+1} \psi_{t+1}) a_t + \theta_{t+1}) \right].
\end{align*}


## Part 2: Solving the problem numerically



### A) Backward iteration and final period
The Euler equation shows that the optimal decision today depends on the optimal decision tomorrow. Luckily, we know that after the final or terminal period, the agent dies and hence the value is 0 beyond death.

When $\mathfrak{v}'_t(a_t) = 0$, the agent wants to consume as much as possible today. Due to the budget constraint, we know that the maximum is the beginning-of-period marginal resources. Therefore $c = m$ and we can construct the value function of the terminal period with that restriction.

Now, we can use this solution for the preiod before and we can calculate all periods by going backward in time.

### B) Calculating expectations
For all periods before the terminal period, we have to calculate expectations, eg. given the end of period assets, what is the beginning-of-next-period market resources and it's implied point on the value function.

Remember that $m_{t+1} = R/(\Gamma_{t+1} \psi_{t+1}) a_t + \theta_{t+1}$ where $\psi$ and $\theta$ are a realisation of a continuous distribution. Meaning, given $a_t$, there are infinitely many realisations of $m_{t+1}$ all being realized with a different probability. To solve this problem, we would need to take the integral:

\begin{align*}
    \mathfrak{v}'_t(a_t) = \int \int \mathfrak{v}'_t(R/(\Gamma_{t+1} \psi) a_t + \theta) dF\psi dF\theta
\end{align*}

As taking integrals is computational intensive, we discretize the income distribution with n equiprobable shockpoints and transofrming the cdf with a step-wise function. This is called the quadrature method of integration.

![Discretization](../../figures/discreteApprox.png)

Now, the problem translates to:

\begin{align*}
    \mathfrak{v}'_t(a_t) = \sum_i \sum_j P(i)*P(j) \mathfrak{v}'_t(R/(\Gamma_{t+1} \psi_{i}) a_t + \theta_{j})
\end{align*}

In non-technical terms: For each point of the a-Grid, we calculate one possible realisation of income to get to the next period market resources and evaluate next period's marginal value function at this point. We do this for all discretized shock points (49) and take their weighted sum.
This caluclation, a computer can solve easily.

### C) EGM

One way to solve for the optimal consumption choice given the state variable $m_t$ is to pick a $c_t$, calculate $a_t$ and evaluate the euler equation. If the equation does not hold, pick a different $c_t$ until convergence.

A much simpler approach is given by EGM for which we use the Euler equation directly. The FOC is:
\begin{align*}
u^{\prime}(c_t) = \mathfrak{v}'_t(a_t)
\end{align*}
We could take the inverse of $u^{\prime}(c_t)$ to get $c_t$. With CRRA utility, we have
\begin{align*}
    u^{\prime}(c_t) = (c_t)^{-\rho}
\end{align*}
And therefore can rewrite the Euler equation:
\begin{align*}
u^{\prime}(c_t) &= \mathfrak{v}'_t(a_t)\\
 (c_t)^{-\rho} &= \mathfrak{v}'_t(a_t)\\
c_t &= (\mathfrak{v}'_t(a_t))^{-1/ \rho}
\end{align*}

In practice, we fix end of period assets, construct $\mathfrak{v}'_t(a_t)$ and calculate the optimal $c_t$ given $a_t$ using EGM. Now, we can use the budget constraint to back out beginning-of-period market resources ($m_t = c_t + a_t$) and construct a policy function $c_t(m_t$)

### D) Constructing Functions
We need to construct policy functions $c_t(m_t)$, value functions $v_t(m_t)$ and marginal value function $v'_t(m_t)$. The problem is that these functions are non-linear and approximating them can be tricky. In practice we use piece-wise linear approximations. That is, we calculate the value of a function for many points and assume the function is linear between them.

1. Policy functions:
The trick here is to choose a grid which has more points at the parts which are more non-linear. This tends to be where the end-of-period assets are small. Therefore, we use an multi-exponential grid for $a$.

2. Value and marginal value functions:
These functions are even more non-linear than policy functions. The trick we use here is to use the functional form of the CRRA utility function.
- For the marginal value function, we take the FOC of the consumption function $v'_t(m_t) = (c_t(m_t))^{- \rho}$.
- For the value Function, we evaluate the inverse of the function which is much more linear ($v^{-1/\rho}$). After interpolating this function, we construct the true function by taking the inverse again. 

3. Adding a limit function:
Another great tip is to use limit functions. We specify our policy functions on an in-sample grid, but often want to evaluate it on an out-of sample grid. Think of simulations when we have very wealthy households. Piece-wise linear approximation assumes linearity between grid-points, it also assumes linearity after the last grid point. As a result, the approximation can become very bad the further we want to evaluate the function from the last grid point. Very often, we know the limit functions! For instance, the example above converges to the perfect foresight solution. If an agent becomes infinitely rich, the proportion of income on market resources becomes a very small. Therefore, the uncertainty of income does not play a role.

### Summary:
To solve this consumption savings model we do the following:
1. Construct a grid of end-of-period assets ($a_t$)
2. Solve the terminal period using the budget constraint
3. Construct functions of policies $c_t(m_t)$ and (marginal) value function $v_t(m_t)$, $v'_t(m_t)$

4. Calculate $\mathfrak{v}'_t(a_t)$ by taking the expectations from $a_t$ to $m_{t+1}$ and evaluate next periods marginal value function
5. Use EGM to get optimal $c_t$ and back out $m_t$ with the budget constraint
6. Construct functions of policies $c_t(m_t)$ and (marginal) value function $v_t(m_t)$, $v'_t(m_t)$
7. Repeat steps 4-6 until first period or until convergence ($|v'_t(m_t)$ - $v'_{t+1}(m_{t+1})|$ < criterion)